# Train Low Test Models

This notebook is a streamlined notebook for generating minima of low test accuracy through three different means:
- Dataset Poisoning
- Adding Noise to Data
- Decreasing Dataset Sizes

## Imports

In [1]:
# Standard library
import copy
import os
import sys
import time

# Third-party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Local package imports
from minima_volume.dataset_funcs import (
    prepare_datasets,
    save_dataset,
    save_model,
)
from minima_volume.train_funcs import evaluate, train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:

# ==============================
# Base Input Parameters
# ==============================
# --- SEEDS ---
data_seed = 1           
model_seed = 1           

# --- Training configuration ---
epochs = 500
batch_sizes = [100, 200, 500, 1000, 6000]

# --- Dataset configuration ---
base_data_size = 6000
dataset_type = "data"   
dataset_quantities = [0]

# --- Output configuration ---
base_output_dir = ""     
save_generated_dataset = True   
save_generated_models = True    


## Model + Dataset Specific Code

This is for specific code.

In [3]:
# User specifies the model module name
from minima_volume.models import MNIST_model_data as model_module

# Generate dataset
x_base, y_base, x_test, y_test = model_module.get_dataset(
    device = device
)

# MNIST specific initialization parameters
hidden_dims = [256, 128]

# Grab model
model_template = model_module.get_model(hidden_dims=hidden_dims, device=device, seed=model_seed)

# Grab loss and metrics
loss_fn = model_module.get_loss_fn()
other_metrics = model_module.get_additional_metrics()

## Training

We generate the various datasets used to train our models here, before training them. We record the losses, and what each model was trained on.

In [4]:
# ==============================
# Prepare datasets
# ==============================
x_base_train, y_base_train, x_additional, y_additional = prepare_datasets(
    x_base=x_base,
    y_base=y_base,
    dataset_type=dataset_type,
    dataset_quantities=dataset_quantities,
    base_data_size=base_data_size,
    data_seed=data_seed,
    seed_1=None,
    seed_2=None,
)

x_base_train = x_base_train.to(device)
y_base_train = y_base_train.to(device)
x_additional = x_additional.to(device)
y_additional = y_additional.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

# ==============================
# Training loop
# ==============================
all_models = []

for batch_size in batch_sizes:
    print (f"Training batch_size {batch_size}")
    # Assemble training dataset
    x_train = x_base_train#, x_additional[:additional_data]], dim=0)
    y_train = y_base_train#, y_additional[:additional_data]], dim=0)

    # Initialize model (defined in the model-specific file)
    torch.manual_seed(model_seed)
    model = copy.deepcopy(model_template)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    #batch_size = len(x_train) # We are varying batch size now!!!

    # Train model
    train_loss, train_other_metrics, test_loss, test_other_metrics = train(
        model = model,
        x_train = x_train, y_train = y_train,
        x_test = x_test, y_test = y_test,
        loss_fn = loss_fn,
        metrics = other_metrics,
        optimizer = optimizer,
        epochs=epochs,
        batch_size=batch_size,
        verbose_every=100,
    )
    
    # Build dictionary dynamically for additional metrics
    train_metrics_dict = {}
    test_metrics_dict = {}
    if train_other_metrics is not None:
        # train_other_metrics is a list of dicts per epoch
        for metric_name in train_other_metrics[0].keys():  # keys from first epoch
            train_metrics_dict[f"train_{metric_name}"] = [m[metric_name] for m in train_other_metrics]
            test_metrics_dict[f"test_{metric_name}"] = [m[metric_name] for m in test_other_metrics]
    
    # Store results
    trained_model = {
        "model": model,
        "train_loss": train_loss,
        "test_loss": test_loss,
        "additional_data": 0,
        "dataset_type": dataset_type,
        "batch_size": batch_size,
        **train_metrics_dict,  # dynamically include additional metrics
        **test_metrics_dict,
    }
    
    all_models.append(trained_model)

    # Free memory (important for large GPU datasets)
    del x_train, y_train
    torch.cuda.empty_cache()


Training batch_size 100
Epoch 1/500: Train Loss 0.7270 | Test Loss 0.3438 | accs Train 0.7888 Test 0.8954
Epoch 100/500: Train Loss 0.0000 | Test Loss 0.2800 | accs Train 1.0000 Test 0.9553
Epoch 200/500: Train Loss 0.0000 | Test Loss 0.3455 | accs Train 1.0000 Test 0.9561
Epoch 300/500: Train Loss 0.0000 | Test Loss 0.4072 | accs Train 1.0000 Test 0.9563
Epoch 400/500: Train Loss 0.0001 | Test Loss 0.2727 | accs Train 1.0000 Test 0.9563
Epoch 500/500: Train Loss 0.0000 | Test Loss 0.3095 | accs Train 1.0000 Test 0.9586
Training batch_size 200
Epoch 1/500: Train Loss 0.9611 | Test Loss 0.3972 | accs Train 0.7282 Test 0.8814
Epoch 100/500: Train Loss 0.0001 | Test Loss 0.2663 | accs Train 1.0000 Test 0.9524
Epoch 200/500: Train Loss 0.0000 | Test Loss 0.3048 | accs Train 1.0000 Test 0.9526
Epoch 300/500: Train Loss 0.0000 | Test Loss 0.3389 | accs Train 1.0000 Test 0.9533
Epoch 400/500: Train Loss 0.0000 | Test Loss 0.3710 | accs Train 1.0000 Test 0.9541
Epoch 500/500: Train Loss 0.0000

Epoch 100/1000: Train Loss 0.1904 | Test Loss 1.2123 | accs Train 0.9000 Test 0.6130


Epoch 200/1000: Train Loss 0.0048 | Test Loss 3.6333 | accs Train 1.0000 Test 0.6172


Epoch 300/1000: Train Loss 0.0005 | Test Loss 4.2644 | accs Train 1.0000 Test 0.6195


Epoch 400/1000: Train Loss 0.0002 | Test Loss 4.5421 | accs Train 1.0000 Test 0.6190


...

Epoch 600/1000: Train Loss 0.0002 | Test Loss 0.0127 | accs Train 1.0000 Test 0.9968


Epoch 700/1000: Train Loss 0.0001 | Test Loss 0.0130 | accs Train 1.0000 Test 0.9968


Epoch 800/1000: Train Loss 0.0001 | Test Loss 0.0134 | accs Train 1.0000 Test 0.9968


Epoch 900/1000: Train Loss 0.0001 | Test Loss 0.0136 | accs Train 1.0000 Test 0.9968


Epoch 1000/1000: Train Loss 0.0000 | Test Loss 0.0139 | accs Train 1.0000 Test 0.9968
Completed training with 700 additional samples of data


## Training Summary

In [5]:
# ====================================
# Summary of Training Results
# ====================================
print("=== True Generalization ===")
for model_data in all_models:
    model = model_data["model"]
    additional_data = model_data["batch_size"]

    test_loss, test_metrics = evaluate(
        model=model,
        x_test=x_test,
        y_test=y_test,
        loss_fn=loss_fn,
        metrics=other_metrics
    )

    metrics_str = " | ".join([f"{name}: {val:.4f}" for name, val in test_metrics.items()])
    print(
        f"{additional_data:>4} samples | "
        f"Test Loss: {test_loss:.4f}" + (f" | {metrics_str}" if metrics_str else "")
    )

print("\n=== Model Diagnostics by Training Data ===")
for additional_data in dataset_quantities:
    # Build dataset with this many additional samples
    x_train = torch.cat([x_base_train, x_additional[:additional_data]], dim=0)
    y_train = torch.cat([y_base_train, y_additional[:additional_data]], dim=0)

    print(f"\nDataset type: {dataset_type}, additional samples: {additional_data}")

    for model_data in all_models:
        model = model_data["model"]
        model_additional_data = model_data["batch_size"]

        train_loss, train_metrics = evaluate(
            model=model,
            x_test=x_train,
            y_test=y_train,
            loss_fn=loss_fn,
            metrics=other_metrics
        )

        metrics_str = " | ".join([f"{name}: {val:.4f}" for name, val in train_metrics.items()])
        print(
            f" Model {model_additional_data:>4} | "
            f"Train Loss: {train_loss:.4f}" + (f" | {metrics_str}" if metrics_str else "")
        )

    # Free memory if large
    del x_train, y_train
    torch.cuda.empty_cache()

=== True Generalization ===
 100 samples | Test Loss: 0.3095 | accs: 0.9586
 200 samples | Test Loss: 0.4030 | accs: 0.9541
 500 samples | Test Loss: 0.3690 | accs: 0.9493
1000 samples | Test Loss: 0.3340 | accs: 0.9478
6000 samples | Test Loss: 0.3227 | accs: 0.9423

=== Model Diagnostics by Training Data ===

Dataset type: data, additional samples: 0
 Model  100 | Train Loss: 0.0000 | accs: 1.0000
 Model  200 | Train Loss: 0.0000 | accs: 1.0000
 Model  500 | Train Loss: 0.0000 | accs: 1.0000
 Model 1000 | Train Loss: 0.0000 | accs: 1.0000
 Model 6000 | Train Loss: 0.0003 | accs: 1.0000


## Model Saving

In [7]:
# ====================================
# Save Datasets and Models
# ====================================
output_folder = "models_and_data"
# Save dataset (Possible to skip)
if save_generated_dataset:
    save_dataset(
        folder=output_folder,
        filename="dataset.pt",
        x_base_train=x_base_train,
        y_base_train=y_base_train,
        x_additional=x_additional,
        y_additional=y_additional,
        x_test=x_test,
        y_test=y_test,
        dataset_quantities=dataset_quantities,
        dataset_type=dataset_type,
    )
    print(f"Saved dataset to {output_folder}/dataset.pt")

# Save trained models
if save_generated_models:
    for model_data in all_models:
        filename = f"model_batch_{model_data['batch_size']}.pt"
        save_model(
            folder=output_folder,
            filename=filename,
            model=model_data["model"],
            train_loss=model_data["train_loss"],
            train_accs=model_data["train_accs"],
            test_loss=model_data["test_loss"],
            test_accs=model_data["test_accs"],
            additional_data=model_data["additional_data"],
            dataset_type=model_data["dataset_type"],
            batch_size=model_data["batch_size"]
        )
        print(f"Saved model: {output_folder}/{filename}")

✅ Dataset saved to models_and_data\dataset.pt
Saved dataset to models_and_data/dataset.pt
✅ Model saved to models_and_data\model_batch_100.pt
Saved model: models_and_data/model_batch_100.pt
✅ Model saved to models_and_data\model_batch_200.pt
Saved model: models_and_data/model_batch_200.pt
✅ Model saved to models_and_data\model_batch_500.pt
Saved model: models_and_data/model_batch_500.pt
✅ Model saved to models_and_data\model_batch_1000.pt
Saved model: models_and_data/model_batch_1000.pt
✅ Model saved to models_and_data\model_batch_6000.pt
Saved model: models_and_data/model_batch_6000.pt
